# Incremental agglomerative clustering
Incremental agglomerative clustering, given old clusters, maps new data to old clusters and creates new clusters for unmapped records. It is a bottom-up approach, meaning it assumes all the data points belong to separate clusters initially. Then it recursively merges the cluster pairs which have minimum distance between them. This kind of approach is useful when we are dealing with temporal text data and need to cluster it incrementally in time. For example, news, social media posts, chats etc. which keep on increasing with time and there is no endpoint to wait for before doing the analysis. This implementation is based on the following paper:

* X. Dai, Q. Chen, X. Wang and J. Xu, "Online topic detection and tracking of financial news based on hierarchical clustering," 2010 International Conference on Machine Learning and Cybernetics, 2010, pp. 3341-3346, doi: 10.1109/ICMLC.2010.5580677.

## Steps
•	Considers set of records, does tf-idf vectorization -> agglomerative hierarchical clustering (sklearn) -> for next interval, update tf-idf vectorizer-> use clusters identified in just previous interval as candidate clusters, perform agglomerative hierarchical clustering on new data -> map/merge new clusters with candidate clusters.

Steps to identify topics for new set of stories, given some candidate topics as previous set of stories and corresponding topic: 

1. Get the set of candidate clusters CTS from previous set.
2. Get the set of new clusters in new set using agglomerative clustering (sklearn).
3. Get a cluster Tc from the new set NTC, and calculate the similarity between Tc and each single cluster ct within the old set CTS. If the maximum similarity, which is the similarity between ct and Tc, is not smaller than the threshold θ, we consider that ct is related to Tc.
4. Combine the cluster Tc into the previous cluster ct, and rebuild the cluster model.
5. Delete Tc from NTC, repeat from step 3.

## Example
This notebook shows how to use this clustering method by running an example with kaggle dataset (https://www.kaggle.com/rmisra/news-category-dataset). This is a news category dataset with date information along with headline and short description. We have shown clustering over headlines only, dataset being sorted according to date.

In [1]:
import pandas as pd
from clustering.agglomerative_clusters import find_clusters
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth=-1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_json(".\\sample_data\\example_news_data.json", lines=True)
df.sort_values(by='date', inplace=True)         # sort the data by date
df['content'] = df['headline']       #  method requires the text 

In [3]:
# first chunk
df_p1 = df[:5000]
out = find_clusters(df_p1, thresh1=0.6, nfeatures=400)   # function call
len(out['class_prd'].unique())

threshold: 0.6, nfeatures: 400


836

In [4]:
out.loc[out.class_prd==237, ['headline']]          # example cluster, it can be noticed that this cluster consists of headlines related "Killing".

,headline
49626,Syrian Boys Cry For Brother Killed In Aleppo Bombing
49509,Cousin Of NBA Star Dwyane Wade Killed In Chicago Shooting
49395,Lightning Bolt Kills More Than 300 Reindeer In Norway
48904,Philippines President Declares 'State Of Lawlessness' After Bombing Kills 14
48273,Tanzania Earthquake Kills Multiple People
47941,The Gazebo Where Tamir Rice Was Killed To Be Displayed At Museum
47732,Sleep Deprivation Is Killing You And Your Career
47654,"Wildlife Services' -- AKA Murder, Inc.'s -- Unregulated Killing Fields: The Body Count of this Killing Agency Is Sickeningly Reprehensible"
47028,Miami Marlins Pitcher Jose Fernandez Killed In Boating Accident
45972,U.S. Researcher Killed By Rock-Throwing Protesters In Ethiopia


In [5]:
# second chunk

df_p2 = df[5000:10000]
out2 = find_clusters(df_p2, thresh1=0.6, nfeatures=400, old_samples=out)

threshold: 0.6, nfeatures: 400


In [6]:
out2[out2.class_prd==237]['headline']     # same cluster name in new chunk. We can observe that the headlines related to "Killing" have been grouped into same cluster as for previous chunk.

index
4999    Laurie Hernandez Killed It During Her 'DWTS' Salsa Performance, Obviously                        
5691    4 Killed On Ride At Australia's Biggest Theme Park                                               
6060    Saudi-Led Raid Kills At Least 60 At Security Site And Prison In Yemen                            
6559    Tracy Morgan Forgives The Truck Driver Who Almost Killed Him                                     
6702    Suicide Bombers In Ambulance Kill At Least 21 People In Iraq                                     
7251    Facebook Temporarily Killed Off A Lot Of Its Users                                               
7268    Taliban Suicide Bomber Kills 4 At NATO Base In Bagram, Afghanistan                               
7791    Fuel Tanker Explosion In Mozambique Kills At Least 73, Government Says                           
7996    India Train Derailment Kills At Least 146 People, Rescuers End Search                            
8692    Massive Wildfire Engulfs Tenness

We can iterate over the whole data in small chunks in similar manner. Feel free to checkout other clusters and to test with your own data.

# 2 Compare Clusters
## map and compare clusters to ground truth clusters using f-measure as the metric
Lets assume ground truth has M number of clusters and clustering result has N number of clusters.

For each m<sup>th</sup> cluster in ground truth, calculate f-measure with every cluster in clustering result. This f-measure indicates how good the cluster C<sub>n</sub> describes the cluster C<sub>m</sub>.

I<sub>mn</sub> → Intersection of elements in m<sup>th</sup> cluster in ground truth and n<sup>th</sup> cluster in predicted clusters.

|C<sub>m</sub>| = number of elements in m<sup>th</sup> cluster

precision p = I<sub>mn</sub>/|C<sub>n</sub>|,            recall r = I<sub>mn</sub>/|C<sub>m</sub>|

F-measure of mth and nth cluster fmn = 2.r.p/(r+p) = 2.I<sub>mn</sub>/(|C<sub>m</sub>|+|C<sub>n</sub>|)

2. Create a matrix with cluster labels in ground truth  as row index, cluster labels in results as column index and f-measures of clusters as values.

3. Identify the cluster pair with maximum f-measure, assume that these clusters are mapped and store these mappings and corresponding f-measures, remove the row and column corresponding to these clusters. Repeat this until we get empty matrix.

4. Overall f-measure is the average of f-measure corresponding to each cluster map identified in previous step. Since while calculating f-measure for each cluster pair, we are dividing Imn by sum of number of elements in both clusters, hence further weights were not added while calculating the mean f-measure.

Relevant References:
* Wagner, Silke, and Dorothea Wagner. Comparing clusterings: an overview. Karlsruhe: Universität Karlsruhe, Fakultät für Informatik, 2007.



## Parameters:
* `df`: *pd.DataFrame*, help=input with ground truth clusters  and predicted clusters
expected columns: 'class_gt' with numeric labels for ground truth clusters or 'text_category' with text
                  'class_prd' with numeric labels for predicted clusters or 'predicted_topic' with text

* `gt_column`: str, help=column in which ground truth text clusters are present, default="text_category"
                    if clusters are numeric, skip this parameter but make sure columns with numeric clusters are named 'class_gt' or 'class_prd'

## Returns:
    -------
* `df`: *pd.DataFrame* with mapped clusters in 'mapped_cluster' and corresponding f-measure in 'max_fmeasure' column
* `fmeasure_aggregate`: *float*
                    average f-measure as over-all performance metric
* `true_dct`: *pd.DataFrame* with clusters which match exactly

**Lets see how does the results from clustering incrementally and in one go using agglomerative clustering compare.**

In [8]:
# combine incremental results (1st 5000, next 5000)
out2.set_index('old_index', inplace=True)
out_incremental = out.append(out2)
# take all 10000 at once
test_df = df[:10000]
test_df = find_clusters(test_df, thresh1=0.6, nfeatures=400)
# join both results
res1 = test_df['class_prd']
res1.name = 'class_gt'
matched_df = pd.DataFrame(res1).join(out_incremental)
len(res1.unique()), len(out_incremental['class_prd'].unique())

threshold: 0.6, nfeatures: 400


(1113, 1167)

In [11]:
from clustering.compare_clusters import cluster_measure
res_df, fmeasure_aggregate, true_matches = cluster_measure(matched_df, gt_column='class_gt')
fmeasure_aggregate, len(true_matches)               # f-measure is 0.7, which should be 1 for perfect match and 0 for no match. len(true_matches) i.e. number of cluster matching perfectly is 245.

(0.7056323066709902, 245)